# Luna Project

We obtained data of the SP500 index, Bitcoin, Ethereum, the weather and calculated the moon phases to determine if there's a correlation between each of these assets and the weather or the moon.

# OBTAINING MOON DATA

In [ ]:
import ephem
from datetime import datetime, timedelta
import pandas as pd

def calculate_moon_phase(date):
    known_new_moon_date = ephem.Date("2014-01-01")  # Replace with an actual known New Moon date
    days_since_new_moon = int(date - known_new_moon_date)
    lunar_cycle = 29.53 # How it was gotten? what is this?
    moon_phase = days_since_new_moon % lunar_cycle

    if moon_phase < 1.845:
        return "New Moon"
    elif moon_phase < 5.555:
        return "Waxing Crescent"
    elif moon_phase < 9.265:
        return "First Quarter"
    elif moon_phase < 12.975:
        return "Waxing Gibbous"
    elif moon_phase < 16.685:
        return "Full Moon"
    elif moon_phase < 20.395:
        return "Waning Gibbous"
    elif moon_phase < 24.105:
        return "Last Quarter"
    elif moon_phase < 27.815:
        return "Waning Crescent"
    else:
        return "New Moon"

# Initialize an empty list to store results
results = []

# Example usage for a range of dates
start_date = datetime(2014, 1, 1)
end_date = datetime(2024, 1, 31)

current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime("%Y-%m-%d")
    moon_phase_result = calculate_moon_phase(ephem.Date(date_str))
    results.append({"Date": date_str, "Moon Phase": moon_phase_result})
    current_date += timedelta(days=1)

# Convert the results list to a pandas DataFrame
df = pd.DataFrame(results)

# Print the DataFrame
print(df)

# Obtaining S&P500 data via AlphaVantage API

In [ ]:
#import requests

#token = "GET YOUR OWN!!!!!"

#url = "https://www.alphavantage.co/query"
#params = {"apikey":token,
        # "function":"TIME_SERIES_DAILY",
        # "symbol":"SPY",
        # "outputsize":"full"}

#r_spy = requests.get(url,params=params)
#r_spy.json().keys()


In [ ]:
#import pandas as pd
#spy_data = pd.DataFrame(r_spy.json()['Time Series (Daily)']).T
#spy_data

# Obtain SPY, BTC, ETH, Moon and NY temp

Data is available for download.
Files are the following:

- btc_nomoon.csv
- moons.csv
- eth_nomoons.csv
- nytemps.csv
- sp500_nomoon.csv


In [ ]:
btc = pd.read_csv("C:\\Users\\PUT YOUR PATH\\OneDrive\\Escritorio\\IRONhack\\Week 3\\projet\\btc_nomoon.csv")
eth = pd.read_csv("C:\\Users\\PUT YOUR PATH\\OneDrive\\Escritorio\\IRONhack\\Week 3\\projet\\eth_nomoons.csv")
sp500 = pd.read_csv("C:\\Users\\PUT YOUR PATH\\OneDrive\\Escritorio\\IRONhack\\Week 3\\projet\\sp500_nomoon.csv")
moons = pd.read_csv("C:\\Users\\PUT YOUR PATH\\OneDrive\\Escritorio\\IRONhack\\Week 3\\projet\\moons.csv")
nytemps = pd.read_csv("C:\\Users\\PUT YOUR path\\OneDrive\\Escritorio\\IRONhack\\Week 3\\projet\\nytemps.csv")

# Obtaining Correlations

In [ ]:
#Adding tickers/lastminute cleaning
#Maybe a function could be usefull to have less repetition

btc["%24H"] = btc["%24H"].abs()
eth["%24H"] = eth["%24H"].abs()

sp500['daily_return']=sp500['daily_return'] * 100
sp500 = sp500.rename(columns={'daily_return': '%24H'})
sp500["%24H"] = sp500["%24H"].abs()
sp500 = sp500.rename(columns={'date': 'Date'})

sp500 = sp500.rename(columns={'%24H': '%Volatility'})
eth = eth.rename(columns={'%24H': '%Volatility'})
btc = btc.rename(columns={'%24H': '%Volatility'})

nytemps = nytemps.rename(columns={'date': 'Date'})
nytemps["avg_temp"] = (nytemps["avg_temp"]-32)*5/9#Converting to celsius
nytemps = nytemps.rename(columns={'avg_temp': 'Temperature/Cº'})

mergedbtc = pd.merge(btc,nytemps,on="Date")
mergedsp500 = pd.merge(sp500,nytemps,on="Date")
mergedeth = pd.merge(eth,nytemps,on="Date")

btc = pd.merge(mergedbtc,moons,on="Date")
eth = pd.merge(mergedeth,moons,on="Date")
sp500 = pd.merge(mergedsp500,moons,on="Date")

btc.set_index("Date",inplace=True)
eth.set_index("Date",inplace=True)
sp500.set_index("Date",inplace=True)

btcorr = btc.corr()
ethcorr = eth.corr()
sp500corr = sp500.corr()

# Clean/order le data

In [ ]:
def clean_btcethdata(x):
    # Drop columns: 'Waxing Gibbous', 'Waxing Crescent' and 9 other columns
    x = x.drop(columns=['Waxing Gibbous', 'Waxing Crescent', 
                                  'Waning Gibbous', 'Waning Crescent', 
                                  'New Moon', 'Last Quarter', 'Full Moon', 
                                  'First Quarter', 'Temperature/Cº', 'Open', 'Close'])
    return x

clean_data(btcorr)  #Where is the function???

In [ ]:
def clean_spydata(x):
    # Drop columns: 'Waxing Gibbous', 'Waxing Crescent' and 9 other columns
    x = x.drop(columns=['Waxing Gibbous', 'Waxing Crescent', 
                                  'Waning Gibbous', 'Waning Crescent', 
                                  'New Moon', 'Last Quarter', 'Full Moon', 
                                  'First Quarter', 'Temperature/Cº', 'closing_price'])
    return x